# Defining the aoi

In [ ]:
import pandas as pd
import numpy as np
import xarray as xa
from pathlib import Path
import src.data.ecostress_io as eio
import rioxarray
import sys
import geopandas as gpd
import json
import dask


root_path = Path("/scratch/rave/")

with open(f"{root_path}/core/geo-countries/archive/countries.geojson", "rb") as f:
    all_countries_geojson = json.loads(f.read())


for i in all_countries_geojson['features']:
    if i['properties']['ADMIN'] == "France":
        france_geo = i


france_gdf = gpd.GeoDataFrame.from_features([france_geo]).explode()

france_gdf[france_gdf.area==max(france_gdf.area)].plot()

aoi = france_gdf[france_gdf.area==max(france_gdf.area)]

aoi_geojson = json.loads(aoi.to_json())

xmin, ymin, xmax, ymax = aoi.total_bounds
bounds_tuple = (4, 42, 7, 47)
xmin, ymin, xmax, ymax = bounds_tuple  # hardcoding since concattenating 1000s of ecostress files with different overlaps hangs

# Global rivers dataset

In [ ]:
rivers_df = gpd.read_file(Path(root_path, "europe_rivers/eu_river.shp"))

rivers_df['R_ID'] = rivers_df['R_ID'].apply(int).apply(str)

france_rivers_df = rivers_df.cx[xmin:xmax, ymin:ymax]

aoi.crs = france_rivers_df.crs # setting crs for aoi

### Check to make sure they overlay, geopandas has poor support for linestrings so we subset by the bounding box

In [ ]:
base = aoi.plot(color="grey", edgecolor="black")

france_rivers_df.plot(ax=base, color="blue")

## reading in paths and example data array

In [ ]:
qa_path = Path(root_path, "rhone-ecostress-data", "ECO3ANCQA")
et_path = Path(root_path, "rhone-ecostress-data", "ECO3ETPTJPL")
esi_path = Path(root_path, "rhone-ecostress-data", "ECO4ESIPTJPL")

whole_tif_qa_paths, csv_qa_paths, xml_qa_paths = eio.separate_extensions(qa_path)

whole_tif_etdaily_paths, csv_et_paths, xml_et_paths = eio.separate_extensions(et_path, "*ETdaily*.tif")

In [ ]:
all_etdaily_paths = clip_and_save(whole_tif_etdaily_paths, bounds_tuple, filter_nan = True, outDir="/scratch/rave/tmp")

In [ ]:
all_et_daily = eio.read_scenes(all_etdaily_paths)

In [ ]:
all_qa_paths = clip_and_save(whole_tif_qa_paths, bounds_tuple, filter_nan = True, outDir="/scratch/rave/tmp")

Need to find a way to merge dask arrays into a single data array or dataset. ideally paying attention to coordinates (requires regridding) but if not, ignoring coordinates so non spatial ops like NaN filtering can occur in parrallel. 

In [ ]:
## ECOSTRESS Availability over the Rhone River
all_qa_daily, all_et_daily = eio.match_da_lists(all_qa_daily, all_et_daily)
date_df = eio.get_date_df(all_et_daily)
# date_df.groupby([date_df['date'].dt.month, date_df['date'].dt.year]).count().plot(kind="bar")

In [ ]:
buffered_france_rivers_df = france_rivers_df.to_crs(epsg=2154)\
                                            .buffer(5000)\
                                            .to_crs(epsg=4326) # buffers by 5000 meters
france_rivers_df['geometry'] = buffered_france_rivers_df

In [ ]:
resolution = all_et_daily[0].rio.resolution()
crs = france_rivers_df.crs
aoi_grid = eio.gdf_to_dataarray(france_rivers_df, crs, resolution)

In [ ]:
resampled_et_paths = resample_and_save(all_et_daily, aoi_grid, outDir="/scratch/rave/tmp")

In [ ]:
import matplotlib.pyplot as plt
from rasterio.enums import Resampling

possibly done did it. xarray plot plots intersect of minimum bound rect?

In [ ]:
import matplotlib.pyplot as plt
from rasterio.enums import Resampling

ax = plt.subplot()

r0 = all_et_daily[0].rio.reproject_match(cube['value'], resampling=Resampling.nearest)
r4 = all_et_daily[4].rio.reproject_match(cube['value'], resampling=Resampling.nearest)

r0.sel(band=1).plot.imshow(ax=ax, cmap="inferno")
r4.sel(band=1).plot.imshow(ax=ax, add_colorbar=False)


In [ ]:
r4.rio.to_raster("/scratch/rave/r4.tif")

In [ ]:
r0.sel(band=1).plot.imshow()

In [ ]:
r0.shape

In [ ]:
r4.shape

In [ ]:
all_et_daily[4].shape

In [ ]:
all_et_daily[0].shape

In [ ]:
ax = plt.subplot()
all_et_daily[0].sel(band=1).plot.imshow(ax=ax, cmap="inferno")
# et0.rio.reproject_match(et10).plot(ax=ax, add_colorbar=False)
# all_et_daily[4].sel(band=1).plot.imshow(ax=ax, add_colorbar=False)

Checking how well a contant lat lon resolution preserves distance/area rleationships over the aoi

https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

trying xesmf takes a long time

In [ ]:
resolution = .0006349

In [ ]:
import geopy.distance

lonmin, latmin, lonmax, latmax = buffered_france_rivers_df.total_bounds
print("resolution distance at lower left", geopy.distance.geodesic((lonmin,latmin),(lonmin+resolution,latmin)).m)
print("resolution distance at upper right", geopy.distance.geodesic((lonmax,latmax),(lonmax,latmax+resolution)).m)

Need to make grid out and use xesmf to convert the grid ins

In [ ]:
buffered_france_rivers_df.unary_union.centroid

xesmf requires working with constant latitude and longitutde resolution.

In [ ]:
import xesmf as xe

grid_out = {"lon": np.arange(lonmin, lonmax, resolution),
           "lat": np.arange(latmin, latmax, resolution)}

In [ ]:
lon, lat = np.meshgrid(all_et_daily[0]['x'], all_et_daily[0]['y'])

In [ ]:
grid_in = {'lon': lon, 'lat': lat}


In [ ]:
help(xe.Regridder)

In [ ]:
regridder = xe.Regridder(grid_in, grid_out, method="bilinear")
regridder.clean_weight_file()
regridder


In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib inline

ETcolors = ["#f6e8c3", "#d8b365", "#99974a", "#53792d", "#6bdfd2", "#1839c5"]
ETcmap = LinearSegmentedColormap.from_list("ET", ETcolors)
date_utc = pd.to_datetime(et['date'].values)
layer_type = et.attrs['filename'].split("_")[-3]
title = 'ECO3ETPTJPL Evapotranspiration'

fig = plt.figure(figsize=(9.7,7.6))                                                       # Set the figure size (x,y)
fig.suptitle(f'{title} ({layer_type}) \n at {date_utc}', fontsize=22)  # Add title for the plots
plt.axis('off')                                                                           # Remove axes from plot
im = plt.imshow(et.sel(band=1), cmap=ETcmap);                                                        # Plot array using colormap
# plt.scatter(Tcol, Trow, color="black", marker='x')                                        # Plot tower location
# Add a colormap legend
plt.colorbar(im, orientation='horizontal', fraction=0.05, pad=0.004, label=f"ET ({et.attrs['units']})", shrink=0.6).outline.set_visible(True)

In [ ]:
xa.open_rasterio(tif_qa_paths[0], parse_coordinates=True)

## Code graveyard

trying to plot ecostress DataArray with geopandas shapes

In [ ]:
def cartopy_project_geo_df(df, crs):

    # This can be converted into a `proj4` string/dict compatible with GeoPandas
    crs_proj4 = crs.proj4_init
    return df.to_crs(crs_proj4)

crs = ccrs.PlateCarree()
aoi_projected = cartopy_project_geo_df(aoi, crs)
france_rivers_df_projected = cartopy_project_geo_df(france_rivers_df, crs)
# base = aoi_projected.plot(color="grey", edgecolor="black")
# france_rivers_df_projected.plot(ax=base, color="blue")


import cartopy.crs as ccrs
import matplotlib.pyplot as plt
ax = plt.axes(projection=crs)
all_et_daily[14].sel(band=1).plot.imshow(ax=ax, transform=crs)
# ax.add_geometries(aoi_projected['geometry'], crs=crs)
ax.add_geometries(france_rivers_df_projected['geometry'], crs=crs)

In [ ]:
def mask_NA_values():
    """
    Daily ET products have both nan values from where there are clouds 
    and -1e+13 for where the ecostress swath was clipped during the ordering process
    """
    masked_et = np.ma.masked_where(et.sel(band=1) == np.nan, et.sel(band=1))
    masked_et = np.ma.masked_where(masked_et == -1e+13, masked_et)